In [1]:
import pandas as pd

In [2]:
data = pd.read_excel(
    'C:/Users/glappe/Documents/validated_phones_and_polos_2021-03-23.xlsx'
)

In [3]:
data.head()

,ME_NO,FNAME,MNAME,LNAME,OFFICE_PHONE,ADDR1,ADDR2,ADDR_CITY,ADDR_STATE,ADDR_ZIP
0,4204880064,CARMEN,J,SIERRA LUCIANO,7872818522,NaN,400 CALLE MANUEL DOMENECH STE 311,SAN JUAN,PR,918
1,3545120088,ERIN,ELIZABETH,CASEY,6364417280,NaN,11 GARVEY PKWY,SAINT CHARLES,MO,63303
2,69007840063,OBIOHA,LOUIS,OKORO,4787466662,NaN,844 2ND ST,MACON,GA,31201
3,65501960025,MOUNIR,NaN,BELCADI,8056526729,NaN,200 HILLMONT AVE,VENTURA,CA,93003
4,802951248,GORDON,SCOTT,ABRAMS,4126411635,MAGEE-WOMENS HOSPITAL,300 HALKET ST,PITTSBURGH,PA,15213


### Get Symphony data

In [5]:
symphony_file = 'U:/Source Files/Data Analytics/Data-Science/Data/Symphony/2021-03-26_Symphony_Data.csv'
symphony = pd.read_csv(symphony_file, dtype=str)


In [6]:
symphony.head()

,SYM_POLO_MAILING_LINE_1,SYM_POLO_MAILING_LINE_2,SYM_POLO_CITY,SYM_POLO_STATE,SYM_POLO_ZIP,SYM_TELEPHONE_ORIG,SYM_FAX_ORIG,SYM_ME,SYM_TELEPHONE_NUMBER,SYM_FAX_NUMBER
0,UNIT 2,10035 HALEDON AVE,DOWNEY,CA,90240,NaN,NaN,0051762743,NaN,NaN
1,STE 300,3401 NORTH BLVD,BATON ROUGE,LA,70806,(225) 767-5520,(225) 767-4934,0051488011,2257675520,2257674934
2,NaN,1783 TROUP HWY,TYLER,TX,75701,(903) 595-2283,(903) 597-7018,0051488014,9035952283,9035977018
3,NaN,2121 SANTA MONICA BLVD,SANTA MONICA,CA,90404,(310) 829-8913,(310) 315-6168,0051488016,3108298913,3103156168
4,NaN,1500 EXPO PKWY,SACRAMENTO,CA,95815,(916) 646-8300,(916) 920-4434,0051488017,9166468300,9169204434


In [9]:
symphony['SYM_ME'].values

array(['0051762743', '0051488011', '0051488014', ..., '5611758011',
       '5611758013', '5611758014'], dtype=object)

In [7]:
data['ME_NO'] = data['ME_NO'].apply(lambda x: ('00000'+str(x))[-11:])
data['OFFICE_PHONE'] = data['OFFICE_PHONE'].apply(lambda x: str(x).strip())

In [10]:
data['ME10'] = data['ME_NO'].apply(lambda x: x[:10])

In [12]:
merge = data[['ME_NO', 'ME10', 'OFFICE_PHONE']].merge(
    symphony[['SYM_ME', 'SYM_TELEPHONE_NUMBER']],
    left_on='ME10',
    right_on='SYM_ME',
    how='left'
)
merge

,ME_NO,ME10,OFFICE_PHONE,SYM_ME,SYM_TELEPHONE_NUMBER
0,04204880064,0420488006,7872818522,0420488006,7872818522
1,03545120088,0354512008,6364417280,0354512008,6364417280
2,69007840063,6900784006,4787466662,6900784006,4787466662
3,65501960025,6550196002,8056526729,6550196002,8056525755
4,00802951248,0080295124,4126411635,0080295124,4129498100
...,...,...,...,...,...
495,49529760051,4952976005,5594551500,4952976005,5594855000
496,04177901658,0417790165,7325367144,0417790165,7325367144
497,00801930336,0080193033,3179639300,0080193033,3177156402
498,01803940262,0180394026,5158759192,0180394026,5158759192


### Coverage

In [15]:
(len(data) - merge['SYM_ME'].isna().sum()) / len(data)

0.994

### Match Rate

Of the phones which this data source has, what proportion of them match what we recently validated?

In [16]:
merge['match'] = merge['OFFICE_PHONE'] == merge['SYM_TELEPHONE_NUMBER']

In [19]:
merge['match'].sum() / len(merge)

0.644

In [20]:
merge[
    ['ME_NO', 'OFFICE_PHONE', 'SYM_TELEPHONE_NUMBER']
].rename(
    columns={'SYM_TELEPHONE_NUMBER': 'SYMPHONY_PHONE'}
).to_csv('symphony_comparison.csv', index=False)